In [1]:
import glob
import os
import subprocess
import glob
import datetime as dt
from pathlib import Path

import numpy as np
import pandas as pd


from mintpy.objects import gnss, timeseries
from mintpy.smallbaselineApp import TimeSeriesAnalysis
from mintpy.utils import ptime, readfile, utils as ut

from matplotlib import pyplot as plt
from urllib.request import urlretrieve
from matplotlib.backends.backend_pdf import PdfPages

import matplotlib.dates as mdates

In [2]:
def standardize_longitude(lon, limit='-180to180'):
    """Normalize the longitude value range into (-180, 180] or [0, 360).

    Parameters: lon   - float / np.ndarray, longitude in degree
                limit - str, -180to180 or 0to360
    Returns:    lon   - float / np.ndarray, longitude in degree
    """
    lon = np.asarray(lon)

    # ensure data within (-180, 360)
    lon = np.where(lon >= 360, lon - 360, lon)
    lon = np.where(lon <= -180, lon + 360, lon)

    # range option 1: ensure data within (-180, 180]
    if limit == '-180to180':
        lon = np.where(lon > 180, lon - 360, lon)

    # range option 2: ensure data within [0, 360)
    elif limit == '0to360' and np.nanmin(lon) < 0:
        lon = np.where(lon < 0, lon + 360, lon)

    return float(lon) if np.isscalar(lon) else lon
    
class GNSS_ESESES:
    """GNSS child class for daily solutions processed for the Enhanced Solid
    Earth Science ESDR System (ESESES) project by JPL and SOPAC.

    Website: https://cddis.nasa.gov/Data_and_Derived_Products/GNSS/ESESES_products.html
             http://garner.ucsd.edu/pub/measuresESESES_products/
    """
    def __init__(self, site: str, data_dir=None, version='IGS14', url_prefix=None):
        self.site = site
        self.data_dir = data_dir or os.getcwd()  # Default to current working directory
        self.version = version
        self.source = 'ESESES'
        self.url_prefix = url_prefix
        

        # get file
        self.file = os.path.join(self.data_dir, f'{self.site.lower():s}CleanTrend.neu.Z')

        # get url
        # moved to GNSS_ESESES.dload_site() to avoid searching url_prefix
        # when downloading is not needed.


    def dload_site(self, overwrite=True, total_tries=5, print_msg=True):
        """Download GNSS data file.
        """
        from zipfile import ZipFile

        # get url
        if not self.url_prefix:
            self.url_prefix = gnss.get_ESESES_url_prefix()
        self.url = os.path.join(self.url_prefix, os.path.basename(self.file)).replace("\\", "/");#print('URL is',self.url);
        if self.url and overwrite or not os.path.isfile(self.file):
            print(f"downloading site {self.site:s} from {self.source} to {self.file:s}")
            # retry on download fail
            # https://stackoverflow.com/questions/31529151
            remain_tries = total_tries
            while remain_tries > 0 :
                try:
                    urlretrieve(self.url, self.file)
                    print(f'successfully downloaded: {self.url}')
                except:
                    print(f'error downloading {self.url} on trial no. {total_tries-remain_tries}')
                    remain_tries -= 1
                    continue
                else:
                    break

        # call parent class to download
        #super().dload_site(overwrite=overwrite, print_msg=print_msg)

        # uncompress the downloaded *.z file [for ESESES only]
        with ZipFile(self.file, 'r') as fz:
            fz.extractall(self.data_dir)
        self.file = self.file.strip('.Z')    # update file name
        if print_msg:
            print(f'... extracted to {self.file:s}')

        return self.file


    def get_site_lat_lon(self, print_msg=False) -> (float, float):
        """Get station lat/lon based on processing source.
        Retrieve data from the displacement file.

        Modifies:   self.lat/lon - float
        Returns:    self.lat/lon - float
        """
        # download file if it does not exist
        if not os.path.isfile(self.file):
            self.dload_site(print_msg=print_msg)

        # use the uncompressed data file
        if self.file.endswith('.Z'):
            self.file = self.file[:-2]

        with open(self.file) as f:
            lines = f.readlines()

            # latitude
            lat_line = [x for x in lines if x.startswith('# Latitude')][0].strip('\n')
            self.site_lat = float(lat_line.split()[-1])

            # longitude
            lon_line = [x for x in lines if x.startswith('# East Longitude')][0].strip('\n')
            self.site_lon = float(lon_line.split()[-1])

        # ensure longitude in the range of (-180, 180]
        self.site_lon = standardize_longitude(self.site_lon, limit='-180to180')

        return self.site_lat, self.site_lon
    def _crop_to_date_range(self, start_date: str, end_date: str):
        """Crop the time-series given the start/end_date in format YYYYMMDD,
        and create date_list from dates.
        """
        flag = np.ones(len(self.dates), dtype=bool)
        if start_date:
            t0 = ptime.date_list2vector([start_date])[0][0]
            flag[self.dates < t0] = 0
        if end_date:
            t1 = ptime.date_list2vector([end_date])[0][0]
            flag[self.dates > t1] = 0

        self.dates = self.dates[flag]
        self.dis_e = self.dis_e[flag]
        self.dis_n = self.dis_n[flag]
        self.dis_u = self.dis_u[flag]
        self.std_e = self.std_e[flag]
        self.std_n = self.std_n[flag]
        self.std_u = self.std_u[flag]

        # create member var: date_list
        self.date_list = [x.strftime('%Y%m%d') for x in self.dates]


    def read_displacement(self, start_date=None, end_date=None, print_msg=True, display=False):
        """Read GNSS displacement time-series (defined by start/end_date).

        Parameters: start/end_date - str, date in YYYYMMDD format
        Returns:    dates          - 1D np.ndarray of datetime.datetime object
                    dis_e/n/u      - 1D np.ndarray of displacement in meters in float32
                    std_e/n/u      - 1D np.ndarray of displacement STD in meters in float32
        """
        vprint = print if print_msg else lambda *args, **kwargs: None

        # download file if it does not exist
        if not os.path.isfile(self.file):
            self.dload_site(print_msg=print_msg)

        # use the uncompressed data file
        if self.file.endswith('.Z'):
            self.file = self.file[:-2]

        # read data file
        # use the first 9 cols only, as some epoches miss 10-13 cols: CorrNE/NU/EU, Chi-Squared
        #print('reading time and displacement in east/north/vertical direction')
        fc = np.loadtxt(self.file, usecols=tuple(range(0,9)))
        num_solution = fc.shape[0]

        # parse dates
        dates = [dt.datetime(int(fc[i, 1]), 1, 1) + dt.timedelta(days=int(fc[i, 2]))
                 for i in range(num_solution)]
        self.dates = np.array(dates)

        # parse displacement data
        (self.dis_n,
         self.dis_e,
         self.dis_u,
         self.std_n,
         self.std_e,
         self.std_u) = fc[:, 3:9].astype(np.float32).T / 1000

        # cut out the specified time range
        self._crop_to_date_range(start_date, end_date)

        # display if requested
        if display:
            self.plot()

        return (self.dates,
                self.dis_e, self.dis_n, self.dis_u,
                self.std_e, self.std_n, self.std_u)


In [3]:
#InSAR_vert_filename = r'C:\Users\bvarugu\Documents\merged_UP_GNSS_corrected_timeseries.h5';
InSAR_vert_filename = r'C:\Users\bvarugu\Documents\merged_UP_GNSS_corrected_timeseries_TRE_GNSS.h5'
insar_ts_label = "DWR Vertical"

date_list = timeseries(InSAR_vert_filename).get_date_list();
insar_dates = ptime.date_list2vector(date_list)[0];
num_date = len(date_list)
date0, date1 = (insar_dates[0]-dt.timedelta(days=15)).strftime('%Y%m%d'), (insar_dates[-1]+dt.timedelta(days=15)).strftime('%Y%m%d')


vert_ts,insar_metadata = readfile.read(InSAR_vert_filename);

coord = ut.coordinate(insar_metadata)

In [4]:
Towill_GNSS_filename = r'C:\Users\bvarugu\Documents\ArcGIS\Projects\InSAR_discussion\Outputs\GNSS_station_network_TRE_Towill.csv'
Towill_GNSS = pd.read_csv(Towill_GNSS_filename,header=0);
Towill_GNSS['site'] = Towill_GNSS['CODE'].str[-4:];
unused_sites= sorted(list(Towill_GNSS['site'][Towill_GNSS['USED']==0]));
used_sites= sorted(list(Towill_GNSS['site'][Towill_GNSS['USED']==1]));


In [5]:
gnss_source = 'ESESES';
filter_by_TRE = True;
(S,N,W,E) = ut.four_corners(insar_metadata)
ymin, ymax = 0, int(insar_metadata['LENGTH'])-1;
xmin, xmax = 0, int(insar_metadata['WIDTH'])-1;

Towill_sites= unused_sites + used_sites;
if filter_by_TRE==True:
    gnss_names,gnss_lats, gnss_lons = gnss.search_gnss(SNWE=(S,N,W,E), source=gnss_source,start_date=date0, end_date=date1);
    in_Towill = [i for i,gnss in enumerate(gnss_names) if gnss in used_sites];
    out_Towill = [i for i,gnss in enumerate(gnss_names) if gnss in unused_sites];
    in_site_names, in_site_lats, in_site_lons = gnss_names[in_Towill],gnss_lats[in_Towill],gnss_lons[in_Towill];
    out_site_names, out_site_lats, out_site_lons = gnss_names[out_Towill],gnss_lats[out_Towill],gnss_lons[out_Towill];
    site_names = np.array(list(in_site_names) + list(out_site_names)); 
else:
    site_names, site_lats, site_lons = gnss.search_gnss(SNWE=(S,N,W,E), source=gnss_source,start_date=date0, end_date=date1);
print('Final number of sites:',len(site_names))

load 3655 GNSS sites with fields: site lon lat
keep sites within SNWE of (32.999169067, 39.0, -123.0, -115.499169667): [797]
['7ODM' 'ACSB' 'AGMT' 'ALPP' 'ALTH' 'ANA1' 'AOA1' 'AR27' 'AR53' 'ARGU'
 'ARM1' 'ARM2' 'ASHM' 'AVRY' 'AZRY' 'AZU1' 'BALD' 'BAR1' 'BATM' 'BBDM'
 'BBRY' 'BCWR' 'BEAT' 'BEMT' 'BEPK' 'BFLD' 'BFSH' 'BGIS' 'BILL' 'BKAP'
 'BKMS' 'BKR1' 'BKR2' 'BLSA' 'BMHL' 'BMRY' 'BOMG' 'BRAN' 'BRI2' 'BRIB'
 'BRPK' 'BSRY' 'BTDM' 'BUEG' 'BULL' 'BUST' 'BVPP' 'CA99' 'CACT' 'CAM5'
 'CAND' 'CAP5' 'CARH' 'CARR' 'CASA' 'CASE' 'CAT1' 'CAT2' 'CAT3' 'CBHS'
 'CCCC' 'CCCO' 'CCCS' 'CCST' 'CDMT' 'CGDM' 'CHAB' 'CHIL' 'CHLO' 'CHMS'
 'CHOW' 'CIRX' 'CIT1' 'CJMG' 'CJMS' 'CJVG' 'CLAR' 'CLBD' 'CMBB' 'CMOD'
 'CMP9' 'CNDR' 'CNPP' 'COPR' 'COSO' 'COTD' 'CPBN' 'CRAM' 'CRAT' 'CRBT'
 'CRCN' 'CRFP' 'CRGG' 'CRHS' 'CRRS' 'CRU1' 'CSCI' 'CSDH' 'CSN1' 'CSST'
 'CTDM' 'CTMS' 'CUHS' 'CVHS' 'DAM1' 'DAM2' 'DAM3' 'DDMN' 'DECH' 'DHLG'
 'DIAB' 'DIXN' 'DLNO' 'DLUZ' 'DOND' 'DONO' 'DREW' 'DS13' 'DSHS' 'DSME'
 'DSSC' 'DUBP' 'DVLE' '

In [6]:
gnss_dir = os.path.join(os.path.dirname(InSAR_vert_filename),'GNSS_ESESES');
os.makedirs(gnss_dir,exist_ok=True)


In [7]:
TRE_GNSS_file_list = glob.glob(r'C:\Users\bvarugu\Documents\ArcGIS\Projects\TRE_Results_view\Outputs\VERT_*_UNR_GNSS_intersect.csv');

dfs = [];
for file in TRE_GNSS_file_list:
    df = pd.read_csv(file,header=0);
    dfs.append(df)
TRE_GNSS = pd.concat(dfs,ignore_index=True);
(S,N,W,E) = ut.four_corners(insar_metadata)
TRE_GNSS = TRE_GNSS[(TRE_GNSS['LAT'] >= S) & (TRE_GNSS['LAT'] <= N) &
                              (TRE_GNSS['LON'] >= W) & (TRE_GNSS['LON'] <= E)];
TRE_GNSS.columns = [col[1:] if col.startswith('D') else col for col in TRE_GNSS.columns];
TRE_datelist = [col for col in TRE_GNSS.columns if col.isdigit()];filtered_TRE_datelist= [col for col in TRE_datelist if date0<= col <=date1];
TRE_GNSS_reduced = TRE_GNSS[filtered_TRE_datelist+['Sta']];
TRE_GNSS_reduced = TRE_GNSS_reduced.reset_index();
TRE_GNSS_grouped = (TRE_GNSS_reduced.groupby('Sta', as_index= False).mean(numeric_only=True))
TRE_dates = ptime.date_list2vector(filtered_TRE_datelist)[0];
TRE_site_names= np.array(TRE_GNSS_grouped['Sta']);

print(len(TRE_site_names))

347


In [8]:
def resample_to_monthly_date(dialy_dis,dialy_dates,monthly_dates):
    
    monthly_dis = []
    
    # Loop over each date
    for date in monthly_dates:
        # Define the ±15-day window around the current InSAR date
        start_date = date - dt.timedelta(days=15)
        end_date = date + dt.timedelta(days=15)
        
        # Find GNSS measurements within the range
        in_window = (dialy_dates >= start_date) & (dialy_dates <= end_date)
        
        # Extract GNSS displacements in the window
        disp_in_window = dialy_dis[in_window]
        
        # Calculate the mean for the window, ignoring NaNs
        mean_displacement = np.nanmean(disp_in_window) if len(in_window) > 0 else np.nan
        
        # Append the mean displacement to the results
        monthly_dis.append(mean_displacement)
    
    return np.array(monthly_dis)


In [41]:
# read the time-series file
plot_unit = 'ft';
if plot_unit == 'ft':
    conversion_fac = 3.28084;
elif plot_unit == 'mm':
    conversion_fac = 1000;
else:
    conversion_fac = 1;

plot_TRE= True;
plot_error_bar = True    

wind=0;
xmin,xmax,ymin,ymax = 0,int(insar_metadata['WIDTH']),0,int(insar_metadata['LENGTH'])

# print(f'reading timeseries from file: {insar_ts_file}')
output_pdf_file = os.path.join(os.path.dirname(InSAR_vert_filename),'GNSSUp_vs_TRE_vs_DWRVert_2022_2024_monthly_{}_new.pdf'.format(plot_unit));
num_site = len(site_names)
prog_bar = ptime.progressBar(maxValue=num_site);
insar_rmse_list = []; tre_rmse_list = [];final_site_names= [];
with PdfPages(output_pdf_file) as pdf:
    fig_count = 0  # To keep track of how many figures are plotted on a page
    fig, axs = plt.subplots(4, 1, figsize=(8, 12), sharex=True)  # Create a 4x1 grid of subplots (4 figures per page)
    for i, site_name in enumerate(site_names):
        prog_bar.update(i+1, suffix=f'{site_name} {i+1}/{num_site}');
       
        gnss_obj = GNSS_ESESES(site =site_name,data_dir=gnss_dir);gnss_obj.get_site_lat_lon();gnss_obj.read_displacement();
        gnss_obj._crop_to_date_range(start_date= date0, end_date= date1);
        gnss_dis = gnss_obj.dis_u;
        gnss_lalo = (gnss_obj.site_lat, gnss_obj.site_lon)
    
        # get relative LOS displacement on common dates
        gnss_dates = gnss_obj.dates
        gnss_monthly_dis = resample_to_monthly_date(gnss_dis,gnss_dates,insar_dates)
        gnss_monthly_dis -= gnss_monthly_dis[0]
    
        # read InSAR
        y, x = coord.geo2radar(gnss_lalo[0], gnss_lalo[1])[:2];
        if y > ymin and y < ymax and x > xmin and x < xmax:
            if wind > 0:
                vert_dis = np.nanmean(vert_ts[:, y-wind:y+wind, x-wind:x+wind],axis=(1,2))
            else:
                vert_dis = vert_ts[:, y, x];
            vert_dis[0] = 0.0;
       # read TRE _data
        if plot_TRE == True:
            if site_name in TRE_site_names:
                station_data = TRE_GNSS_grouped[TRE_GNSS_grouped['Sta'] == site_name];
                TRE_dis = station_data[filtered_TRE_datelist].values.flatten();
                TRE_monthly_dis = resample_to_monthly_date(TRE_dis,np.array(TRE_dates),insar_dates)
                TRE_monthly_dis = TRE_monthly_dis -TRE_monthly_dis[0];
                TRE_monthly_dis = TRE_monthly_dis/1000;
            else:
                TRE_monthly_dis = np.zeros([len(insar_dates)]) * np.nan
            
    
        ## plot figure
        if sum(~np.isnan(gnss_monthly_dis)) > 5 and sum(~np.isnan(vert_dis)) > 5 and sum(~np.isnan(TRE_monthly_dis)) > 5:
            ax = axs[fig_count % 4]  # Access the current subplot in the 4x1 grid
            #save_fig_name = os.path.join(gps_plot_dir,'GNSS_{}_vs_INSAR_LOS.png'.format(site_name));
            ax.scatter(insar_dates, gnss_monthly_dis*conversion_fac, s=2**2, label="GNSS Daily Vertical", color='black');
            gnss_indices = [0, len(insar_dates) // 2, -1]  # Indices for first, middle, last points
            gnss_errors = [0.1, 0.1, 0.1]  # Example error values, adjust as needed
            if plot_error_bar == True:
                for idx, err in zip(gnss_indices, gnss_errors):
                    if not np.isnan(gnss_monthly_dis[idx]):
                        ax.errorbar(insar_dates[idx],gnss_monthly_dis[idx] * conversion_fac,yerr=err,fmt='o',color='black',capsize=3,alpha=0.2)
            #plot DWR INSAR vertical
            ax.scatter(insar_dates, vert_dis*conversion_fac, label=insar_ts_label,marker='o', s=50, edgecolor='blue', facecolors='none');
            if plot_TRE == True:
                ax.scatter(insar_dates, TRE_monthly_dis*conversion_fac, label="TRE Vertical", marker='x', s=50, color='red');
            #ax.scatter(insar_dates, ifg_gnss_corr_dis*1000, label="IFG+GNSS+Corr", s=25, color='red',alpha=0.5);
            insar_rmse = np.sqrt(np.nanmean(((vert_dis*conversion_fac) -(gnss_monthly_dis*conversion_fac)) ** 2));
            tre_rmse = np.sqrt(np.nanmean(((TRE_monthly_dis*conversion_fac) - (gnss_monthly_dis*conversion_fac)) ** 2));
            insar_rmse_list.append(insar_rmse); tre_rmse_list.append(tre_rmse);final_site_names.append(site_name)
            # axis format
            if site_name in used_sites:
                ax.set_title(f"Station Name: {site_name} (used)") 
            elif site_name in unused_sites:
                ax.set_title(f"Station Name: {site_name} (unused)") 
            else:
                ax.set_title(f"Station Name: {site_name}") 
            ax.set_ylabel('LOS displacement [{}]'.format(plot_unit));
            ylim = int(np.ceil((max(abs(vert_dis))*1000)/100))*100;
            if plot_unit == 'ft':
                ylim = ylim * (conversion_fac/1000)
            
            ax.set_ylim([-ylim,ylim]);
            
             # Slant x-axis labels to 45 degrees
            # Set x-ticks at every 4th month
            #start_date = pd.to_datetime("2022-04-01")  # Start from April 2022
            ax.text(0.05, 0.95, f'DWR RMSE: {insar_rmse:.3f}{plot_unit}',transform=ax.transAxes, fontsize=10, verticalalignment='top')
            ax.text(0.05, 0.85, f'TRE RMSE: {tre_rmse:.3f}{plot_unit}', transform=ax.transAxes, fontsize=10, verticalalignment='top')
    
            ax.xaxis.set_major_locator(mdates.MonthLocator(bymonthday=1, interval=3))  # Every 4 months
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))  # Format as Month Year
            plt.xticks(rotation=45)  # Rotate x-axis labels for readability
            
            #plt.setp(ax.get_xticklabels(), rotation=45, ha="right")  
            fig_count += 1
            if fig_count % 4 == 1:
                ax.legend();

            # Save to PDF every 4 plots and reset figure
            if fig_count % 4 == 0:
                pdf.savefig(fig)  # Save current page to PDF
                plt.close(fig)    # Close the figure to free memory
                fig_count = 0
                fig, axs = plt.subplots(4, 1, figsize=(8, 12), sharex=True)  # Create new 4x1 figure for next 4 plots
    if fig_count > 0:
        pdf.savefig(fig)
        plt.close(fig)
prog_bar.close()
#plt.show()


[>                        2%                       ] BEMT 11/672    0s /    20s

C:\Users\bvarugu\AppData\Local\Temp\ipykernel_16212\1669315176.py:18: RuntimeWarning: Mean of empty slice
  mean_displacement = np.nanmean(disp_in_window) if len(in_window) > 0 else np.nan
C:\Users\bvarugu\AppData\Local\Temp\ipykernel_16212\1669315176.py:18: RuntimeWarning: Mean of empty slice
  mean_displacement = np.nanmean(disp_in_window) if len(in_window) > 0 else np.nan


[>                        2%                       ] BGIS 15/672    0s /    36s

C:\Users\bvarugu\AppData\Local\Temp\ipykernel_16212\1669315176.py:18: RuntimeWarning: Mean of empty slice
  mean_displacement = np.nanmean(disp_in_window) if len(in_window) > 0 else np.nan


[=>                       3%                       ] BUEG 23/672    1s /    36s

...

C:\Users\bvarugu\AppData\Local\Temp\ipykernel_16212\1669315176.py:18: RuntimeWarning: Mean of empty slice
  mean_displacement = np.nanmean(disp_in_window) if len(in_window) > 0 else np.nan


[======================= 97% ===================>  ] SOMT 655/672   30s /     0s

C:\Users\bvarugu\AppData\Local\Temp\ipykernel_16212\1669315176.py:18: RuntimeWarning: Mean of empty slice
  mean_displacement = np.nanmean(disp_in_window) if len(in_window) > 0 else np.nan
C:\Users\bvarugu\AppData\Local\Temp\ipykernel_16212\1669315176.py:18: RuntimeWarning: Mean of empty slice
  mean_displacement = np.nanmean(disp_in_window) if len(in_window) > 0 else np.nan


[==================================================] WWFG 672/672   30s /     0s

C:\Users\bvarugu\AppData\Local\Temp\ipykernel_16212\1669315176.py:18: RuntimeWarning: Mean of empty slice
  mean_displacement = np.nanmean(disp_in_window) if len(in_window) > 0 else np.nan
C:\Users\bvarugu\AppData\Local\Temp\ipykernel_16212\1669315176.py:18: RuntimeWarning: Mean of empty slice
  mean_displacement = np.nanmean(disp_in_window) if len(in_window) > 0 else np.nan


In [43]:
insar_rmse_used = []; tre_rmse_used = [];insar_rmse_unused = []; tre_rmse_unused = [];
for i,site in enumerate(final_site_names):
    if site in used_sites:
        insar_rmse_used.append(insar_rmse_list[i]);
        tre_rmse_used.append(tre_rmse_list[i]);
    elif site in unused_sites:
        insar_rmse_unused.append(insar_rmse_list[i]);
        tre_rmse_unused.append(tre_rmse_list[i]);
    else:
        print(f'{site} not in both used and unused lists')
print(f'Used sited RMSE, DWR:{np.mean(insar_rmse_used)},TRE:{np.mean(tre_rmse_used)}')
print(f'Unused sited RMSE, DWR:{np.mean(insar_rmse_unused)},TRE:{np.mean(tre_rmse_unused)}')

Used sited RMSE, DWR:0.021098319956963013,TRE:0.026758052721386077
Unused sited RMSE, DWR:0.0390745744651585,TRE:0.14555653414320616


In [45]:
len(insar_rmse_used)

108

In [47]:
len(insar_rmse_unused)

17